In [5]:
import pandas as pd
import numpy as np

X1 = pd.read_csv('../data/features.csv', header=None).values
X2 = pd.read_csv('../data/raw_images.csv', header=None).values
y1 = pd.read_csv('../data/labels.csv', header=None).values.ravel().astype(int)

print('X1.shape:', X1.shape)
print('X2.shape:', X2.shape)
print('y1.shape:', y1.shape)

X1.shape: (4559, 512)
X2.shape: (4559, 30000)
y1.shape: (4559,)


In [6]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = \
    train_test_split(X1, y1, test_size=0.3, random_state=123, shuffle=True, stratify=y1)

X1_train_sub, X1_valid, y1_train_sub, y1_valid = \
    train_test_split(X1_train, y1_train, test_size=0.2, random_state=123, stratify=y1_train)

print('Train/Valid/Test sizes:', y1_train.shape[0], y1_valid.shape[0], y1_test.shape[0])

Train/Valid/Test sizes: 3191 639 1368


In [7]:
X2_train, X2_test, y1_train, y1_test = \
    train_test_split(X2, y1, test_size=0.3, random_state=123, shuffle=True, stratify=y1)

X2_train_sub, X2_valid, y1_train_sub, y1_valid = \
    train_test_split(X2_train, y1_train, test_size=0.2, random_state=123, stratify=y1_train)

print('Train/Valid/Test sizes:', y1_train.shape[0], y1_valid.shape[0], y1_test.shape[0])

Train/Valid/Test sizes: 3191 639 1368


## XGBoost

### X1

In [11]:
import scipy.stats
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=123, use_label_encoder=False)

params = {
    'n_estimators':[30, 50, 100, 300, 500],
    'min_child_weight':[4,5], 
    "reg_lambda": scipy.stats.uniform(1e-8, 1.0),
    "alpha": scipy.stats.uniform(1e-8, 1.0),
    'gamma':[i/10.0 for i in range(3,6)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gbtree', 'gblinear'],
    'eval_metric': ['rmse'],
    'eta': [i/10.0 for i in range(3,6)],
}

search1 = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=params,
    n_iter=15,
    cv=5,
    verbose=2,
    n_jobs=1,
    random_state=123)

search1.fit(X1_train, y1_train)

search1.best_score_

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END alpha=0.6964691955978617, booster=gbtree, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=100, objective=reg:tweedie, reg_lambda=0.7800277719120792, subsample=0.7; total time=   2.8s
[CV] END alpha=0.6964691955978617, booster=gbtree, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=100, objective=reg:tweedie, reg_lambda=0.7800277719120792, subsample=0.7; total time=   2.7s
[CV] END alpha=0.6964691955978617, booster=gbtree, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=100, objective=reg:tweedie, reg_lambda=0.7800277719120792, subsample=0.7; total time=   2.8s
[CV] END alpha=0.6964691955978617, booster=gbtree, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=100, objective=reg:tweedie, reg_

[CV] END alpha=0.7770044488897805, booster=gblinear, colsample_bytree=0.8, eta=0.5, eval_metric=rmse, gamma=0.5, max_depth=4, min_child_weight=4, n_estimators=300, objective=reg:tweedie, reg_lambda=0.8933891731171348, subsample=1.0; total time=   1.3s
[22:50:06] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { colsample_bytree, gamma, max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV] END alpha=0.5018366858843366, booster=gblinear, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.5, max_depth=7, min_child_weight=5, n_estimators=300, objective=reg:squarederror, reg_lambda=0.2504553753965067, subsample=0.9; total time=   1.3s

[CV] END alpha=0.545068016466465, booster=gblinear, colsample_bytree=0.6, eta=0.3, eval_metric=rmse, gamma=0.4, max_depth=6, min_child_weight=5, n_estimators=300, objective=reg:tweedie, reg_lambda=0.5104223474780112, subsample=0.7; total time=   1.4s
[22:50:26] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { colsample_bytree, gamma, max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV] END alpha=0.545068016466465, booster=gblinear, colsample_bytree=0.6, eta=0.3, eval_metric=rmse, gamma=0.4, max_depth=6, min_child_weight=5, n_estimators=300, objective=reg:tweedie, reg_lambda=0.5104223474780112, subsample=0.7; total time=   1.4s
[22:50

[CV] END alpha=0.21229811338998222, booster=gblinear, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.4, max_depth=2, min_child_weight=5, n_estimators=30, objective=reg:tweedie, reg_lambda=0.15895965414472274, subsample=0.9; total time=   0.2s
[22:50:39] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { colsample_bytree, gamma, max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV] END alpha=0.21229811338998222, booster=gblinear, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.4, max_depth=2, min_child_weight=5, n_estimators=30, objective=reg:tweedie, reg_lambda=0.15895965414472274, subsample=0.9; total time=   0.2s
[2

[CV] END alpha=0.13089496066408074, booster=gbtree, colsample_bytree=0.8, eta=0.3, eval_metric=rmse, gamma=0.3, max_depth=3, min_child_weight=4, n_estimators=300, objective=reg:tweedie, reg_lambda=0.5594873855040602, subsample=1.0; total time=  11.6s
[22:53:43] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { colsample_bytree, gamma, max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV] END alpha=0.31678790711837973, booster=gblinear, colsample_bytree=0.7, eta=0.5, eval_metric=rmse, gamma=0.3, max_depth=3, min_child_weight=5, n_estimators=50, objective=reg:squarederror, reg_lambda=0.578551478108833, subsample=0.7; total time=   0.3s
[

0.9197712923307872

In [12]:
search1.best_params_

{'alpha': 0.6917018087001772,
 'booster': 'gbtree',
 'colsample_bytree': 0.7,
 'eta': 0.3,
 'eval_metric': 'rmse',
 'gamma': 0.3,
 'max_depth': 7,
 'min_child_weight': 4,
 'n_estimators': 100,
 'objective': 'reg:tweedie',
 'reg_lambda': 0.029423743551983135,
 'subsample': 0.7}

In [14]:
print(f"Training Accuracy: {search1.best_estimator_.score(X1_train, y1_train)*100: 0.2f}%") 
print(f"Test Accuracy: {search1.best_estimator_.score(X1_test, y1_test)*100: 0.2f}%")

Training Accuracy:  99.94%
Test Accuracy:  91.74%


In [24]:
from sklearn.metrics import f1_score
print(f"F1 score: {f1_score(y1_test, search1.best_estimator_.predict(X1_test), average='weighted')}")
      

F1 score: 0.9170036919863896


### X2

In [66]:
search2 = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=params,
    n_iter=15,
    cv=5,
    verbose=10,
    n_jobs=1,
    random_state=123)

search2.fit(X2_train, y1_train)

search2.best_score_

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5; 1/15] START criterion=entropy, max_depth=9, min_samples_split=3........
[02:10:51] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:11:01] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 1/15] END criterion=entropy, m

[02:52:36] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 4/5; 2/15] END criterion=gini, max_depth=None, min_samples_split=4;, score=0.937 total time= 4.9min
[CV 5/5; 2/15] START criterion=gini, max_depth=None, min_samples_split=4........
[02:57:22] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases

[CV 2/5; 4/15] END criterion=gini, max_depth=11, min_samples_split=4;, score=0.934 total time= 5.3min
[CV 3/5; 4/15] START criterion=gini, max_depth=11, min_samples_split=4..........
[03:38:47] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:38:57] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior

[04:21:41] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:21:48] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 6/15] END criterion=gini, max_depth=None, min_samples_split=2;, score=0.936 total time= 4.9min
[CV 2/5; 6/15] START criterion=gini, max_depth=None, min_samples_split=2.....

[05:00:52] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 4/5; 7/15] END criterion=entropy, max_depth=None, min_samples_split=2;, score=0.937 total time= 4.8min
[CV 5/5; 7/15] START criterion=entropy, max_depth=None, min_samples_split=2.....
[05:05:35] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above ca

...

[06:30:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[06:30:10] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 11/15] END criterion=gini, max_depth=7, min_samples_split=2;, score=0.941 total time= 5.0min
[CV 2/5; 11/15] START criterion=gini, max_depth=7, min_samples_split=2.........

[07:10:19] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 4/5; 12/15] END criterion=entropy, max_depth=7, min_samples_split=3;, score=0.928 total time= 5.0min
[CV 5/5; 12/15] START criterion=entropy, max_depth=7, min_samples_split=3.......
[07:15:10] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above case

[CV 2/5; 14/15] END criterion=gini, max_depth=11, min_samples_split=2;, score=0.934 total time= 5.3min
[CV 3/5; 14/15] START criterion=gini, max_depth=11, min_samples_split=2.........
[07:56:15] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { criterion, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[07:56:24] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavio

[08:35:58] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9439072610515058

In [67]:
search2.best_params_

{'min_samples_split': 4, 'max_depth': None, 'criterion': 'gini'}

In [68]:
print(f"Training Accuracy: {search2.best_estimator_.score(X2_train, y1_train)*100: 0.2f}%") 
print(f"Test Accuracy: {search2.best_estimator_.score(X2_test, y1_test)*100: 0.2f}%")

Training Accuracy:  100.00%
Test Accuracy:  94.81%


In [69]:
from sklearn.metrics import f1_score
print(f"F1 score: {f1_score(y1_test, search2.best_estimator_.predict(X2_test), average='weighted')}")

F1 score: 0.9480100060014328


## Logistic Regression

### X1

In [40]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=123, max_iter = 1000)

params = {
    "C": scipy.stats.expon(scale=.01),
    "fit_intercept": [True, False],
    "warm_start": [True,False]
}

search3 = RandomizedSearchCV(
    estimator=lr,
    param_distributions=params,
    n_iter=15,
    cv=10,
    verbose=2,
    n_jobs=1,
    random_state=123)

search3.fit(X1_train, y1_train)

search3.best_score_

Fitting 10 folds for each of 15 candidates, totalling 150 fits
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.1s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.1s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.1s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time=   0.0s
[CV] END C=0.002572840801170508, fit_intercept=

[CV] END C=0.007583288393287821, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.007583288393287821, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.007583288393287821, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.007583288393287821, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C=0.010062180612636784, fit_intercept=True, warm_start=False; total time=   0.0s
[CV] END C

0.759635579937304

In [41]:
search3.best_params_

{'C': 0.03950982068814718, 'fit_intercept': True, 'warm_start': False}

In [42]:
print(f"Training Accuracy: {search3.best_estimator_.score(X1_train, y1_train)*100: 0.2f}%") 
print(f"Test Accuracy: {search3.best_estimator_.score(X1_test, y1_test)*100: 0.2f}%")

Training Accuracy:  76.68%
Test Accuracy:  78.22%


In [43]:
from sklearn.metrics import f1_score
print(f"F1 score: {f1_score(y1_test, search3.best_estimator_.predict(X1_test), average='weighted')}")


F1 score: 0.7691277689909498


### X2

In [44]:
search4 = RandomizedSearchCV(
    estimator=lr,
    param_distributions=params,
    n_iter=15,
    cv=5,
    verbose=2,
    n_jobs=1,
    random_state=123)

search4.fit(X2_train, y1_train)

search4.best_score_

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time= 1.7min
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time= 1.9min
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time= 1.7min
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time= 1.5min
[CV] END C=0.011922721434811058, fit_intercept=True, warm_start=True; total time= 1.7min


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END C=0.002572840801170508, fit_intercept=True, warm_start=False; total time= 1.9min


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END C=0.002572840801170508, fit_intercept=True, warm_start=False; total time= 1.8min
[CV] END C=0.002572840801170508, fit_intercept=True, warm_start=False; total time= 1.7min
[CV] END C=0.002572840801170508, fit_intercept=True, warm_start=False; total time= 1.7min
[CV] END C=0.002572840801170508, fit_intercept=True, warm_start=False; total time= 1.7min


...

[CV] END C=0.004490857897120136, fit_intercept=False, warm_start=True; total time= 1.7min


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END C=0.004490857897120136, fit_intercept=False, warm_start=True; total time= 1.6min
[CV] END C=0.0034773509004921363, fit_intercept=True, warm_start=False; total time= 1.5min
[CV] END C=0.0034773509004921363, fit_intercept=True, warm_start=False; total time= 1.6min
[CV] END C=0.0034773509004921363, fit_intercept=True, warm_start=False; total time= 1.5min
[CV] END C=0.0034773509004921363, fit_intercept=True, warm_start=False; total time= 1.6min
[CV] END C=0.0034773509004921363, fit_intercept=True, warm_start=False; total time= 1.6min


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9288631825785784

In [46]:
search4.best_params_

{'C': 0.002572840801170508, 'fit_intercept': True, 'warm_start': False}

In [47]:
print(f"Training Accuracy: {search4.best_estimator_.score(X2_train, y1_train)*100: 0.2f}%") 
print(f"Test Accuracy: {search4.best_estimator_.score(X2_test, y1_test)*100: 0.2f}%")

Training Accuracy:  100.00%
Test Accuracy:  93.20%


In [48]:
from sklearn.metrics import f1_score
print(f"F1 score: {f1_score(y1_test, search4.best_estimator_.predict(X2_test), average='weighted')}")

F1 score: 0.931976655005432


## Decision Tree + Bagging

### X1

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

tree = DecisionTreeClassifier(random_state=123)

params =  {
    'min_samples_split': [2, 3, 4],
    'max_depth': [5, 7, 9, 11, None],
    'criterion':['entropy', 'gini']
}

search5 = RandomizedSearchCV(
    estimator=tree,
    param_distributions=params,
    n_iter=15,
    cv=5,
    verbose=2,
    n_jobs=1,
    random_state=123)

search5.fit(X1_train_sub, y1_train_sub)

search5.best_score_

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END criterion=entropy, max_depth=9, min_samples_split=3; total time=   0.3s
[CV] END criterion=entropy, max_depth=9, min_samples_split=3; total time=   0.3s
[CV] END criterion=entropy, max_depth=9, min_samples_split=3; total time=   0.3s
[CV] END criterion=entropy, max_depth=9, min_samples_split=3; total time=   0.3s
[CV] END criterion=entropy, max_depth=9, min_samples_split=3; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_split=4; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_split=4; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_split=4; total time=   0.4s
[CV] END criterion=gini, max_depth=None, min_samples_split=4; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_split=4; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, min_samples_split=4; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, min_sam

0.8691232109282069

In [50]:
search5.best_params_

{'min_samples_split': 3, 'max_depth': 5, 'criterion': 'gini'}

In [51]:
print(f"Training Accuracy: {search5.best_estimator_.score(X1_train_sub, y1_train_sub)*100: 0.2f}%") 
print(f"Valid Accuracy: {search5.best_estimator_.score(X1_valid, y1_valid)*100: 0.2f}%")

Training Accuracy:  91.77%
Valid Accuracy:  87.32%


In [53]:
from sklearn.ensemble import BaggingClassifier

bag1 = BaggingClassifier(base_estimator=search5.best_estimator_, 
                        n_estimators=150, 
                        oob_score=True, 
                        bootstrap=True,
                        bootstrap_features=False, 
                        n_jobs=1, 
                        random_state=123)

bag1.fit(X1_train_sub, y1_train_sub)
print(f"Train Accuracy: {bag1.score(X1_train_sub, y1_train_sub)*100:0.2f}%")
print(f"Valid Accuracy: {bag1.score(X1_valid, y1_valid)*100:0.2f}%")
print(f"Test Accuracy: {bag1.score(X1_test, y1_test)*100:0.2f}%")

Train Accuracy: 93.06%
Valid Accuracy: 88.58%
Test Accuracy: 89.18%


In [57]:
from sklearn.metrics import f1_score
print(f"F1 score: {f1_score(y1_test, bag1.predict(X1_test), average='weighted')}")

F1 score: 0.8912916961773778


### X2

In [61]:
search6 = DecisionTreeClassifier(max_depth=18, 
                                 min_impurity_decrease=0.017492913234125385,
                                 min_samples_split=2)

In [64]:
from sklearn.ensemble import BaggingClassifier

bag2 = BaggingClassifier(base_estimator=search6, 
                        n_estimators=50, 
                        oob_score=True, 
                        bootstrap=True,
                        bootstrap_features=False, 
                        n_jobs=1, 
                        random_state=123)

bag2.fit(X2_train_sub, y1_train_sub)
print(f"Train Accuracy: {bag2.score(X2_train_sub, y1_train_sub)*100:0.2f}%")
print(f"Valid Accuracy: {bag2.score(X2_valid, y1_valid)*100:0.2f}%")
print(f"Test Accuracy: {bag2.score(X2_test, y1_test)*100:0.2f}%")

Train Accuracy: 90.99%
Valid Accuracy: 90.30%
Test Accuracy: 90.20%


In [65]:
from sklearn.metrics import f1_score
print(f"F1 score: {f1_score(y1_test, bag2.predict(X2_test), average='weighted')}")

F1 score: 0.9018573316204965
